In [145]:
import pandas as pd
import requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm
import chromedriver_binary

# link = 'https://www.va.gov/ogc/apps/accreditation/index.asp?'

# driver = webdriver.Chrome()
# driver.get(link)
# driver.encoding = 'utf-8'  # Set encoding to utf-8


In [146]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
# Initialize WebDriver
# driver = webdriver.Chrome()

options = Options()
options.add_argument("--headless")  # Run Chrome in background (no window)
options.add_argument("--disable-gpu")  # (optional) helps on Windows
options.add_argument("--window-size=1920,1080")  # (optional) for full-page loading

driver = webdriver.Chrome(options=options)
# ==================================================
# Open the main page
driver.get("https://www.va.gov/ogc/apps/accreditation/index.asp?")
time.sleep(2)

# Select the Attorney radio button
attorney_radio = driver.find_element(By.XPATH, '//*[@id="frmAccredPeople"]/table[1]/tbody/tr/td[1]/label/input')
attorney_radio.click()
time.sleep(1)

# Get all dropdown options
state_dropdown = Select(driver.find_element(By.XPATH, '//*[@id="State"]'))
all_states = [option.text for option in state_dropdown.options if option.get_attribute("value") != '']

print(f"🌎 Total States to process: {len(all_states)}")

# Prepare list to store data
all_data = []

# Loop through each state by name
for state_name in all_states:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="State"]'))
        )
        state_dropdown = Select(driver.find_element(By.XPATH, '//*[@id="State"]'))

        attorney_radio = driver.find_element(By.XPATH, '//*[@id="frmAccredPeople"]/table[1]/tbody/tr/td[1]/label/input')
        attorney_radio.click()
        time.sleep(1)

        state_dropdown.select_by_visible_text(state_name)
        print(f"\n➡️ Processing State: {state_name}")
        time.sleep(1)

        search_button = driver.find_element(By.XPATH, '//*[@id="frmAccredPeople"]/p[3]/input[1]')
        search_button.click()

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//table[@class="aspbox"]'))
        )
        time.sleep(2)

        a_tags = driver.find_elements(By.XPATH, '//table[@class="aspbox"]//a')

        for i in range(len(a_tags)):
            try:
                a_tags = driver.find_elements(By.XPATH, '//table[@class="aspbox"]//a')
                a_tags[i].click()

                time.sleep(1.5)

                # -------- Scrape the data silently --------
                p_tag = driver.find_element(By.XPATH, '//*[@id="content-area-template-B"]/p[2]').get_attribute('innerHTML')
                lines = p_tag.split('<br>')
                lines = [line.strip() for line in lines if line.strip()]

                full_name = organization = address = phone = email = ""

                if len(lines) > 0:
                    full_name = lines[0]
                if len(lines) > 1:
                    organization = lines[1]

                address_lines = []
                for line in lines[2:]:
                    if '@' in line:
                        email = line.replace('<a href="mailto:', '').replace('">', '').replace('</a>', '').strip()
                    elif any(char.isdigit() for char in line) and "-" in line:
                        phone = line
                    else:
                        address_lines.append(line)

                address = ", ".join(address_lines)

                # Append data into list
                all_data.append({
                    "State": state_name,
                    "Full Name": full_name,
                    "Organization": organization,
                    "Address": address,
                    "Phone": phone,
                    "Email": email
                })
                # ------------------------------------------

            except Exception as e:
                print(f"⚠️ Error while scraping attorney details: {e}")

            driver.back()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//table[@class="aspbox"]'))
            )
            time.sleep(1)

        print(f"✅ Finished scraping State: {state_name}")

        try:
            main_button = driver.find_element(By.XPATH, '//*[@id="content-area-template-B"]/form[1]/input')
            main_button.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="frmAccredPeople"]/table[1]/tbody/tr/td[1]/label/input'))
            )
            time.sleep(2)

        except Exception as e:
            print(f"⚠️ Error returning to main page: {e}")
            break

    except Exception as e:
        print(f"⚠️ Error processing state {state_name}: {e}")
        continue

# After all states
print("\n🏁 All States Scraped Successfully!")
driver.quit()

# Save to Excel
df = pd.DataFrame(all_data)
df.to_excel("attorney_data.xlsx", index=False)
print("📄 Data saved to 'attorney_data.xlsx' successfully!")


🌎 Total States to process: 61

➡️ Processing State: AK: Alaska
✅ Finished scraping State: AK: Alaska

➡️ Processing State: AL: Alabama
✅ Finished scraping State: AL: Alabama

➡️ Processing State: AR: Arkansas
✅ Finished scraping State: AR: Arkansas

➡️ Processing State: AZ: Arizona
✅ Finished scraping State: AZ: Arizona

➡️ Processing State: CA: California
✅ Finished scraping State: CA: California

➡️ Processing State: CO: Colorado
✅ Finished scraping State: CO: Colorado

➡️ Processing State: CT: Connecticut
✅ Finished scraping State: CT: Connecticut

➡️ Processing State: CZ: Canal Zone
✅ Finished scraping State: CZ: Canal Zone

➡️ Processing State: DC: District Of Columbia
✅ Finished scraping State: DC: District Of Columbia

➡️ Processing State: DE: Delaware
✅ Finished scraping State: DE: Delaware

➡️ Processing State: EU: Europe
✅ Finished scraping State: EU: Europe

➡️ Processing State: FC: Foreign Country
✅ Finished scraping State: FC: Foreign Country

➡️ Processing State: FL: Flor